In [1]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import evaluate,train,prepare
from models.NRMS import NRMSModel

In [2]:
hparams = {
    'scale':'demo',
    'name':'nrms',
    'batch_size':5,
    'title_size':20,
    'his_size':50,
    'npratio':4,
    'dropout_p':0.2,
    'query_dim':200,
    'embedding_dim':300,
    'value_dim':16,
    'head_num':16,
    'epochs':1,
    'metrics':'auc,group_auc,mean_mrr,ndcg@5,ndcg@10',
    'device':'cuda:0',
    'attrs': ['title'],
    'k':-1,
    'select':None,
    'save_step':0,
    'save_each_epoch':False,
    'train_embedding':False,
    'mode':'train',
    'news_id':False,
    'validate':False,
}

In [3]:
device = torch.device(hparams['device'])
vocab, loaders = prepare(hparams)

In [4]:
nrmsModel = NRMSModel(vocab=vocab,hparams=hparams).to(device)

In [ ]:
train(nrmsModel, hparams, loaders, interval=10)

In [20]:
nrmsModel.cdd_size = 1

In [18]:
from sklearn.metrics import roc_auc_score
from tqdm import tqdm

def cal_metric(labels, preds, metrics):
    """Calculate metrics,such as auc, logloss.
    """
    res = {}
    for metric in metrics:
        if metric == "auc":
            auc = np.mean(
                [
                    roc_auc_score(each_labels, each_preds)
                    for each_labels, each_preds in zip(labels, preds)
                ]
            )
            res["auc"] = round(auc, 4)
        elif metric == "rmse":
            rmse = mean_squared_error(np.asarray(labels), np.asarray(preds))
            res["rmse"] = np.sqrt(round(rmse, 4))
        elif metric == "logloss":
            # avoid logloss nan
            preds = [max(min(p, 1.0 - 10e-12), 10e-12) for p in preds]
            logloss = log_loss(np.asarray(labels), np.asarray(preds))
            res["logloss"] = round(logloss, 4)
        elif metric == "acc":
            pred = np.asarray(preds)
            pred[pred >= 0.5] = 1
            pred[pred < 0.5] = 0
            acc = accuracy_score(np.asarray(labels), pred)
            res["acc"] = round(acc, 4)
        elif metric == "f1":
            pred = np.asarray(preds)
            pred[pred >= 0.5] = 1
            pred[pred < 0.5] = 0
            f1 = f1_score(np.asarray(labels), pred)
            res["f1"] = round(f1, 4)
        elif metric == "mean_mrr":
            mean_mrr = np.mean(
                [
                    mrr_score(each_labels, each_preds)
                    for each_labels, each_preds in zip(labels, preds)
                ]
            )
            res["mean_mrr"] = round(mean_mrr, 4)
        elif metric.startswith("ndcg"):  # format like:  ndcg@2;4;6;8
            ndcg_list = [1, 2]
            ks = metric.split("@")
            if len(ks) > 1:
                ndcg_list = [int(token) for token in ks[1].split(";")]
            for k in ndcg_list:
                ndcg_temp = np.mean(
                    [
                        ndcg_score(each_labels, each_preds, k)
                        for each_labels, each_preds in zip(labels, preds)
                    ]
                )
                res["ndcg@{0}".format(k)] = round(ndcg_temp, 4)
        elif metric.startswith("hit"):  # format like:  hit@2;4;6;8
            hit_list = [1, 2]
            ks = metric.split("@")
            if len(ks) > 1:
                hit_list = [int(token) for token in ks[1].split(";")]
            for k in hit_list:
                hit_temp = np.mean(
                    [
                        hit_score(each_labels, each_preds, k)
                        for each_labels, each_preds in zip(labels, preds)
                    ]
                )
                res["hit@{0}".format(k)] = round(hit_temp, 4)
        else:
            raise ValueError("not define this metric {0}".format(metric))
    return res

def run_eval(model,dataloader,interval):
    """ making prediction and gather results into groups according to impression_id, display processing every interval batches

    Args:
        model(torch.nn.Module)
        dataloader(torch.utils.data.DataLoader): provide data

    Returns:
        impression_id: impression ids after group
        labels: labels after group.
        preds: preds after group.

    """
    preds = []
    labels = []
    imp_indexes = []
    
    for i,batch_data_input in tqdm(enumerate(dataloader)):
        
        preds.extend(model.forward(batch_data_input).tolist())
        print(model.forward(batch_data_input).shape)
        label = batch_data_input['labels'].squeeze(dim=-1).tolist()
        labels.extend(label)
        imp_indexes.extend(batch_data_input['impression_index'])
    
    all_keys = list(set(imp_indexes))
    all_keys.sort()
    group_labels = {k: [] for k in all_keys}
    group_preds = {k: [] for k in all_keys}

    for l, p, k in zip(labels, preds, imp_indexes):
        group_labels[k].append(l)
        group_preds[k].append(p)

    all_labels = []
    all_preds = []

    for k in all_keys:
        all_labels.append(group_labels[k])
        all_preds.append(group_preds[k])
    
    return group_labels.keys(), all_labels, all_preds

def evaluate(model,hparams,dataloader,interval=100):
    """Evaluate the given file and returns some evaluation metrics.
    
    Args:
        model(nn.Module)
        hparams(dict)
        dataloader(torch.utils.data.DataLoader): provide data
        interval(int): within each epoch, the interval of steps to display loss

    Returns:
        dict: A dictionary contains evaluation metrics.
    """
    hparam_list = ['name','scale','epochs','save_step','train_embedding','select','integrate','his_size','k','query_dim','value_dim','head_num']
    param_list = ['query_words','query_levels']
    model.eval()
    model.cdd_size = 1
    imp_indexes, labels, preds = run_eval(model,dataloader,interval)
    res = cal_metric(labels,preds,model.metrics.split(','))
    print("evaluation results:{}".format(res))
    with open('performance.log','a+') as f:
        # model_name = '{}-{}_{}_epoch{}_step{}_[hs={},topk={}]:'.format(hparams['name'],hparams['select'],hparams['scale'], str(hparams['epochs']), str(hparams['save_step']), str(hparams['his_size']), str(hparams['k']))
        # f.write(model_name + '\n')
        d = {}
        for k,v in hparams.items():
            if k in hparam_list:
                d[k] = v
        for name, param in model.named_parameters():
            if name in param_list:
                d[name] = tuple(param.shape)

        f.write(str(d)+'\n')
        f.write(str(res) +'\n')
        f.write('\n')        
    return res